# BioExplorer - CCFv3
![](../bioexplorer_ccfv3_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3, MovieMaker
from tqdm import tqdm
import os

url = 'localhost:5000'
be = BioExplorer(url)
mm = MovieMaker(be)
core = be.core_api()

cell_radius = 5.0

### Load Atlas

In [ ]:
population_name = 'atlas_ccfv3a_averaged'

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = population_name

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
cerebellum_ids = [
       512,      1025,       519,      1033,       528,      1041,      1049,
      1056,      1064,      1073,      1091,       846,        91, 589508455,
       989,      1143,      1144,      1145,       645,       912,       920,
       928,       936,       944,     10672,     10673,     10674,     10675,
     10676,     10677,       951,     10680,     10679,     10678,     10681,
     10682,       957,     10683,     10687,     10688,     10689,     10690,
     10691,     10692,     10684,     10686,       968,       976,     10705,
     10706,     10707,     10708,     10709,     10710,     10711,     10712,
     10713,       984,     10714,     10715,     10716,     10717,     10718,
       992,     10720,     10721,     10722,     10719,     10723,     10724,
     10725,     10728,      1001,     10726,     10727,     10729,     10730,
     10731,     10733,     10732,     10734,     10737,     10736,     10735,
      1007,      1017,     10685
]

cerebellum_granular_layer_ids = [10675, 10678, 10708, 10711, 10684, 10720, 10723, 10690, 10726, 10705, 10735, 10687, 10681, 10729, 10672, 10732]
cerebellum_molecular_layer_ids =  [10677, 10680, 10710, 10713, 10686, 10722, 10725, 10692, 10728, 10707, 10737, 10689, 10683, 10731, 10674, 10734]


olfactory_bulb_ids = [507, 212, 228, 236, 244, 220]

medula_ids = [
      773,       781,      1039,       789,      1048,        45,      1069,
      560,       307,        53,       568,        61,       576,       69,
      839,      1098,        76,        77,       83,      1107,       852,
      859,       607,        96,       354, 589508451,       101,       106,
      112,       370,       372,       887,       379,       640,      386,
      642,       903,       135,       136,       651,       395,       653,
      143,       659,       661,       666,       154,       161,       674,
      169,       682,       938,       939,       429,       177,       691,
      437,       185,       955,       445,       701,       193,       963,
      711,       970,       203,       202,       206,       207,       720,
      209,       978,       217,       222,       225,       995,       230,
      235,       765, 
      1557651847
]

fibers_ids = [960, 1000, 1009, 396, 109, 698]
regions_to_ignore = [
    1557651848, # Cell excluded from region 1557651847 (x<540)
    1024543562, 2358040414, 3263488087, 2416897036,
    3034756217, 2165415682, 2614168502, 1842735199,
    3101970431, 1140764290, 3092369320, 1811993763]

regions_to_add = set(cerebellum_ids + cerebellum_granular_layer_ids + cerebellum_molecular_layer_ids + olfactory_bulb_ids + medula_ids)
regions_to_ignore = set(fibers_ids + regions_to_ignore)

other_regions_ids = list()
with Session(engine) as session:
    data = session.execute('SELECT guid FROM %s.region WHERE level>=6' % db_schema)
    for d in data.all():
        region_id = int(d[0])
        if region_id not in regions_to_add and region_id not in regions_to_ignore:
          regions_to_add.add(region_id)

In [ ]:
atlas_center = [7075.026185864174,3862.47588645537,5699.969406561653]

In [ ]:
regions = list()
with Session(engine) as session:
    data = session.execute('SELECT DISTINCT(region_guid) FROM %s.cell' % population_name)
    for d in data.all():
        regions.append(int(d[0]))
regions = list(set(regions) - set(regions_to_ignore))

In [ ]:
status = be.start_model_loading_transaction()
status = core.set_application_parameters(image_stream_fps=0)
status = be.reset_scene()
errors = list()
for hemisphere_id in range(2):
    for region_id in tqdm(regions):
        try:
            filter = 'region_guid=%d AND z>%f' % (region_id, atlas_center[2])
            if hemisphere_id == 1:
                filter = 'region_guid=%d AND z<=%f' % (region_id, atlas_center[2])

            atlas_assembly_name = '%d_%d' % (region_id, hemisphere_id)
            be.remove_assembly(atlas_assembly_name)
            atlas_assembly = be.add_assembly(atlas_assembly_name)
            atlas_model = be.add_atlas(
                assembly_name=atlas_assembly_name,
                population_name=population_name,
                load_cells=True, load_meshes=False, cell_radius=cell_radius,
                cell_sql_filter=filter
            )
        except Exception as e:
            errors.append(e)
status = be.commit_model_loading_transaction()
status = core.set_application_parameters(image_stream_fps=20)

In [ ]:
region_colors = dict()

def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) / 256.0 for i in range(0, lv, lv // 3))

with Session(engine) as session:
    data = session.execute('select guid, color_hex_triplet from %s.region' % 'atlas_ccfv3a')
    for d in data.all():
        region_colors[int(d[0])] = hex_to_rgb(d[1])

In [ ]:
model_ids = be.get_model_ids()['ids']
for model_id in model_ids:
    model_name = be.get_model_name(model_id)['name']
    region_id =  int(model_name.split('_')[0])
    c = [1, 0, 0] # Error color
    if region_id in region_colors:
        c = region_colors[region_id]
    alpha = 1.0
    color = [alpha * c[0], alpha * c[1], alpha * c[2]]

    material_ids = be.get_material_ids(model_id)['ids'][:-1]
    palette = list()
    opacities = list()
    shading_modes = list()
    specular_exponents = list()
    user_params = list()
    refraction_indices = list()
    glossinesses = list()
    reflection_indices = list()
    for material_id in material_ids:
        opacities.append(1.0)
        shading_modes.append(be.shading_mode.NONE)
        user_params.append(0.0001)
        glossinesses.append(1.0)
        specular_exponents.append(5.0)
        reflection_indices.append(0.0)
        refraction_indices.append(0.95)
        palette.append(color)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        shading_modes=shading_modes, user_parameters=user_params,
        opacities=opacities, specular_exponents=specular_exponents,
        reflection_indices=reflection_indices,
        refraction_indices=refraction_indices, glossinesses=glossinesses,
        diffuse_colors=palette, specular_colors=palette)

In [ ]:
background_color = [0.0, 0.0, 0.0]
status = core.set_renderer(
    head_light=True,
    current='advanced', background_color=background_color,
    subsampling=4, max_accum_frames=128)
params = core.AdvancedRendererParams()
params.gi_ray_length = 1e6
params.shadow_intensity = 1.0
params.soft_shadow_strength = 0.5
params.main_exposure = 1.5
params.max_ray_depth = 1
params.epsilon_multiplier = 50.0
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

In [ ]:
model_ids = be.get_model_ids()['ids']
nb_models = len(model_ids)
model_center = Vector3()

model_bounds = dict()
for model_id in model_ids:
    bounds = be.get_model_bounds(model_id)
    model_bounds[model_id] = bounds
    model_center.x += bounds.center.x
    model_center.y += bounds.center.y
    model_center.z += bounds.center.z
model_center.x /= nb_models
model_center.y /= nb_models
model_center.z /= nb_models

In [50]:
def explode(factor, speed):
    import math
    model_ids = be.get_model_ids()['ids']
    status = core.set_renderer(subsampling=64, max_accum_frames=0)
    status = core.set_application_parameters(image_stream_fps=0)
    for model_id in model_ids:
        aabb = model_bounds[model_id]

        normal = Vector3(aabb.center.x - model_center.x, aabb.center.y - model_center.y, aabb.center.z - model_center.z)
        length = math.sqrt(normal.x*normal.x + normal.y*normal.y + normal.z*normal.z)
        normal.x /= length
        normal.y /= length
        normal.z /= length

        transformation={
            'rotation': [0.0, 0.0, 0.0, 1.0],
            'rotation_center': [0.0, 0.0, 0.0],
            'scale': [1.0, 1.0, 1.0],
            'translation': [
                normal.x * speed * factor, 
                normal.y * speed * factor,
                normal.z * speed * factor
            ]
        }
        core.update_model(model_id, transformation=transformation)
    status = core.set_renderer(subsampling=4, max_accum_frames=64)
    status = core.set_application_parameters(image_stream_fps=20)    

In [ ]:
core.set_camera(
    orientation=[0.8744458160476791, 0.09151682620478456, -0.401564627390352, 0.25634943991934056],
    position=[-2565.3701241987646, -2607.8377454106976, -1731.3329308640486],
    target=[6733.589672442965, 4796.273454159725, 6016.635720470601],
    current='perspective'
)

In [47]:
output_folder = '/scratch/videos/atlas/%s/explosion/v1' % population_name
os.makedirs(output_folder, exist_ok=True)
k = 4
spp = 64
image_size = [k*960, k*540]

In [48]:
camera_keys = [
    {
        'apertureRadius': 0.0,
        'direction': [0.0, 1.0, 0.0],
        'focalDistance': 1.0,
        'origin': [7075.004011154174, -8914.854648733952, 5699.994689941412],
        'up': [0.0, 0.0, 1.0]
    }
    ,
    {
        'apertureRadius': 0.0,
        'direction': [0.0, 1.0, 0.0],
        'focalDistance': 9834.88625865745,
        'origin': [6771.526760062025, -27624.77339430373, 5775.864168719532],
        'up': [0.0, 0.0, 1.0]
    }    
]

In [51]:
mm.build_camera_path(camera_keys, 100, 1)
nb_frames = mm.get_nb_frames()
for frame in tqdm(range(nb_frames)):
    factor = float(frame) / float(nb_frames)
    mm.set_current_frame(frame)
    explode(factor, 10000)
    mm.create_snapshot(
        renderer='advanced', path=output_folder,
        base_name='%05d' % frame,
        size=image_size, samples_per_pixel=spp)

 89%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏               | 90/101 [16:25:23<2:00:26, 656.93s/it]


RequestError: Socket connection closed